# Miami Hotel Search Engine Demo

This notebook demonstrates the semantic search functionality for finding Miami hotels based on natural language queries, enhanced with real-time internet search results.

In [33]:
# Import required libraries
import sys
sys.path.append('../src')
import importlib
import search
importlib.reload(search)  # Reload the module to get latest changes
from search import HotelSearchEngine
import pandas as pd
from IPython.display import display, Markdown

# Install and import DuckDuckGo search
!pip install -q duckduckgo-search
from duckduckgo_search import DDGS

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Initialize the Search Engine

First, let's create an instance of our search engine and load the hotel data.

In [34]:
# Initialize the search engine
search_engine = HotelSearchEngine('../data/miami_hotels.csv')

# Display basic statistics about our dataset
print(f"Total number of hotels: {len(search_engine.hotels_df)}\n")
print("Available columns in the dataset:")
print(search_engine.hotels_df.columns.tolist())
print("\nSample of available hotels:")
display(search_engine.hotels_df[['name', 'type', 'rating', 'priceRange', 'category']].head())

Available columns in CSV: ['id', 'type', 'name', 'image', 'awards', 'rankingPosition', 'priceLevel', 'priceRange', 'category', 'rating', 'hotelClass', 'hotelClassAttribution', 'phone', 'address', 'email', 'amenities', 'numberOfRooms', 'prices', 'latitude', 'longitude', 'webUrl', 'website', 'rankingString', 'rankingDenominator', 'numberOfReviews', 'review', 'title']
Total number of hotels: 2511

Available columns in the dataset:
['id', 'type', 'name', 'image', 'awards', 'rankingPosition', 'priceLevel', 'priceRange', 'category', 'rating', 'hotelClass', 'hotelClassAttribution', 'phone', 'address', 'email', 'amenities', 'numberOfRooms', 'prices', 'latitude', 'longitude', 'webUrl', 'website', 'rankingString', 'rankingDenominator', 'numberOfReviews', 'review', 'title']

Sample of available hotels:


,name,type,rating,priceRange,category
0,Faena Miami Beach,HOTEL,4.5,"$729 - $1,426",hotel
1,Faena Miami Beach,HOTEL,4.5,"$729 - $1,426",hotel
2,Faena Miami Beach,HOTEL,4.5,"$729 - $1,426",hotel
3,Faena Miami Beach,HOTEL,4.5,"$729 - $1,426",hotel
4,Faena Miami Beach,HOTEL,4.5,"$729 - $1,426",hotel


## Search Results with Internet Information

In [35]:
def render_hotel_markdown(hotel, news, reviews):
    """Format hotel information and search results as markdown"""
    # Hotel info as markdown table
    hotel_md = "### 🏨 Hotel Data\n\n"
    hotel_md += "| Field | Value |\n|---|---|\n"
    for k, v in hotel.items():
        hotel_md += f"| {k} | {str(v).replace('|', '\|')} |\n"

    # News section
    news_md = "\n### 📰 News\n"
    if news:
        for n in news:
            link = f"[{n['title']}]({n.get('url', '')})" if n.get('url') else n['title']
            date = n.get('date', '')
            body = n.get('body', '')
            news_md += f"- **{date}**: {link}\n  \n  {body}\n"
    else:
        news_md += "_No news found._\n"

    # Reviews/info section
    reviews_md = "\n### 💬 Reviews / Info\n"
    if reviews:
        for r in reviews:
            link = f"[{r['title']}]({r.get('href', '')})" if r.get('href') else r['title']
            body = r.get('body', '')
            reviews_md += f"- {link}\n  \n  {body}\n"
    else:
        reviews_md += "_No reviews/info found._\n"
    
    reviews_md += "\n---\n"  # Add separator between hotels
    
    display(Markdown(hotel_md + news_md + reviews_md))

def search_hotels(query, top_k=3):
    """Search for hotels and get internet information"""
    print(f"🔍 Search Query: '{query}'\n")
    
    # Get local search results
    results = search_engine.search(query, top_k=top_k)
    if not results:
        print("No results found.")
        return
        
    # Display results with internet information
    for hotel in results:
        with DDGS() as ddgs:
            news = list(ddgs.news(
                f'"{hotel["name"]}" Miami hotel',
                region='us-en',
                max_results=2
            ))
            reviews = list(ddgs.text(
                f'"{hotel["name"]}" Miami hotel reviews site:tripadvisor.com OR site:booking.com',
                region='us-en',
                max_results=2
            ))
        render_hotel_markdown(hotel, news, reviews)

<>:7: SyntaxWarning: invalid escape sequence '\|'
<>:7: SyntaxWarning: invalid escape sequence '\|'
/var/folders/r9/t2slgqdd6cqcw5n37nm7n5w00000gp/T/ipykernel_59806/2281037899.py:7: SyntaxWarning: invalid escape sequence '\|'
  hotel_md += f"| {k} | {str(v).replace('|', '\|')} |\n"


## Try Some Example Searches

In [38]:
# Example 1: Search for luxury beachfront hotels
search_hotels("luxury beachfront hotels with ocean view", top_k=2)

🔍 Search Query: 'luxury beachfront hotels with ocean view'



### 🏨 Hotel Data

| Field | Value |
|---|---|
| name | Majestic Hotel South Beach |
| type | HOTEL |
| similarity_score | 0.7509213089942932 |
| rating | 3.5 |
| hotel_class | 3.0 |
| price_level | $$ |
| price_range | $134 - $244 |
| address | 660 Ocean Dr, Miami Beach, FL 33139-6219 |
| amenities | [] |
| review | Good locations facing the ocean. If you want a calm night sleep, book a room at the back of the hotel. Harry, at the front desk, is a nice person, committed to help. Rooms at the back are bigger and more comfortable. Breakfast is ok for US beach hotels. |
| number_of_reviews | 169 |
| ranking | #156 of 235 |
| phone | 13054553270 |
| website | http://www.majesticsouthbeach.com |

### 📰 News
- **2025-04-26T06:54:00+00:00**: [Majestic Hotel South Beach Trademark Col](https://www.travelweekly.com/Hotels/Miami-Beach-FL/Majestic-Hotel-South-Beach-Trademark-Col-p58581025)
  
  Giving more advice is better than noneAs long as you know what you're talking about, do not hesitate to give as much advice as you think the client needs. CLIA: Regulate us. Please!Cruise lines ...

### 💬 Reviews / Info
- [734 Verified Hotel Reviews of Majestic Hotel South Beach | Booking.com](https://www.booking.com/reviews/us/hotel/majestic-south-beach.html)
  
  Reviews of Majestic Hotel South Beach This rating is a reflection of how the property compares to the industry standard when it comes to price, facilities and services available. It's based on a self-evaluation by the property. Use this rating to help choose your stay! 660 Ocean Drive, Miami Beach, FL 33139, United States
- [Majestic Hotel South Beach, Miami Beach (updated prices 2025)](https://www.booking.com/hotel/us/majestic-south-beach.html)
  
  Located in Miami Beach's Art Deco district, and directly opposite the beach, this boutique hotel offers 3 on-site dining options and 24-hour concierge services. A flat-screen cable TV, and all-natural toiletries are featured in every room Majestic Hotel South Beach. Select rooms have a full or partial ocean or city view. This South Beach Majestic Hotel also offers VIP passes to local area ...

---


### 🏨 Hotel Data

| Field | Value |
|---|---|
| name | Best Western Plus Atlantic Beach Resort |
| type | HOTEL |
| similarity_score | 0.7202815413475037 |
| rating | 3.5 |
| hotel_class | 3.0 |
| price_level | $$$ |
| price_range | $167 - $267 |
| address | 4101 Collins Ave, Miami Beach, FL 33140-3210 |
| amenities | [] |
| review | Suite was wonderful.  Lots of space. Hotel was clean and pool bar food was good and reasonably priced.  Pool chairs were limited.  Definitely check with gentleman on beach for chair/umbrella rental.  Price was better than what the hotel stated.  |
| number_of_reviews | 2793 |
| ranking | #151 of 235 |
| phone | 18005688520 |
| website | https://www.bestwestern.com/de_DE/book/hotel-rooms.10366.html?iata=00171890&ssob=BLBWI0003G&cid=BLBWI0003G:yext:website:10366 |

### 📰 News
_No news found._

### 💬 Reviews / Info
- [Best Western Plus Atlantic Beach Resort - Tripadvisor](https://www.tripadvisor.com/Hotel_Review-g34439-d224009-Reviews-Best_Western_Plus_Atlantic_Beach_Resort-Miami_Beach_Florida.html)
  
  Book Best Western Plus Atlantic Beach Resort, Miami Beach on Tripadvisor: See 2,912 traveler reviews, 1,204 candid photos, and great deals for Best Western Plus Atlantic Beach Resort, ranked #150 of 212 hotels in Miami Beach and rated 3 of 5 at Tripadvisor.
- [1833 Verified Reviews of Best Western Plus Atlantic Beach Resort ...](https://www.booking.com/reviews/us/hotel/atlantic-beach.html)
  
  Reviews of Best Western Plus Atlantic Beach Resort This rating is a reflection of how the property compares to the industry standard when it comes to price, facilities and services available. It's based on a self-evaluation by the property. Use this rating to help choose your stay! 4101 Collins Avenue, Miami Beach, FL 33140, United States

---


In [41]:
# Example 2: Search for family-friendly hotels with pools
search_hotels("family friendly hotels with swimming pools and activities for kids", top_k=2)

🔍 Search Query: 'family friendly hotels with swimming pools and activities for kids'



### 🏨 Hotel Data

| Field | Value |
|---|---|
| name | Marriott Stanton South Beach |
| type | HOTEL |
| similarity_score | 0.72697913646698 |
| rating | 4.5 |
| hotel_class | 4.0 |
| price_level | $$$$ |
| price_range | $299 - $524 |
| address | 161 Ocean Drive, Miami Beach, FL 33139-7212 |
| amenities | [] |
| review | Great experience and excellent location (with really nice restaurants nearby). My kids really enjoyed the pool and the beach! I would definitely come back!!! 
I would say this is both good for families with kids and also just couples. |
| number_of_reviews | 4377 |
| ranking | #17 of 235 |
| phone | 18446310595 |
| website | https://www.marriott.com/en-us/hotels/miamb-marriott-stanton-south-beach/overview/?scid=45f93f1b-bd77-45c9-8dab-83b6a417f6fe&y_source=1_MjA5Mzk2MC03NjktbG9jYXRpb24ud2Vic2l0ZQ%3D%3D |

### 📰 News
- **2017-06-05T18:00:00+00:00**: [Marriott Stanton South Beach](https://travel.usnews.com/Hotels/review-Marriott_Stanton_South_Beach-Miami_Beach-Florida-131714/)
  
  Guests loved the location of the hotel: it's close enough to ... when choosing to bed down here. The Marriott Stanton South Beach is located in Miami Beach's Art Deco District, less than a ...
- **2023-08-14T04:52:00+00:00**: [Marriott Stanton South Beach GDS Codes](https://www.travelweekly.com/Hotels/Miami-Beach-FL/Marriott-Stanton-South-Beach-p7906995)
  
  across from the demure Century and two doors down from the cheekier Bentley Beach Hotel, this is a smaller and less expensive version of Loews or the Palms, with a full-service format and a fine ...

### 💬 Reviews / Info
- [Marriott Stanton South Beach, Miami Beach (updated prices 2025)](https://www.booking.com/hotel/us/marriott-south-beach.html)
  
  Marriott Stanton South Beach (Hotel), Miami Beach (US) Deals Overview. Info & prices. Amenities. House rules. The fine print. Guest reviews (507) More; Reserve We Price Match. ... Excellent location - rated 9.2/10!(score from 507 reviews) Real guests • Real stays • Real opinions
- [523 Verified Hotel Reviews of Marriott Stanton South Beach - Booking.com](https://www.booking.com/reviews/us/hotel/marriott-south-beach.html)
  
  Reviews of Marriott Stanton South Beach This rating is a reflection of how the property compares to the industry standard when it comes to price, facilities and services available. It's based on a self-evaluation by the property. Use this rating to help choose your stay! 161 Ocean Drive, Miami Beach, FL 33139, United States

---


### 🏨 Hotel Data

| Field | Value |
|---|---|
| name | The Local House |
| type | HOTEL |
| similarity_score | 0.6686521172523499 |
| rating | 4.5 |
| hotel_class | 3.0 |
| price_level | $$$ |
| price_range | $211 - $382 |
| address | 400 Ocean Dr, Miami Beach, FL 33139-6614 |
| amenities | [] |
| review | I really liked this hotel. It’s location is great - a block of a nice beach access, quiet neighborhood three blocks from the non stop party that is the Art Deco district. Restaurant located downstairs is a great place for drinks, breakfast etc.  What would make me love it was some improvements to the pool area. There are not enough loungers. They have a dry bar that is not staffed, so you have to go downstairs to get drinks poolside. There is couches and tables set up there that art not being utilized. It would make more sense to fill it with more lounge chairs, get rid of the bar, and extend room service to the pool. My 2 cents. |
| number_of_reviews | 782 |
| ranking | #46 of 235 |
| phone | 13055385529 |
| website | https://localhouse.com |

### 📰 News
_No news found._

### 💬 Reviews / Info
- [The Local House, Miami Beach (updated prices 2025) - Booking.com](https://www.booking.com/hotel/us/sense-south-beach.en-gb.html)
  
  Concierge services offered during front desk hours. The Local House is the on-site restaurant at the The Local House. They offer an international tapas menu, brunch, and a full service bar. This Miami Beach hotel is steps from art galleries, boutique shopping and famous restaurants on Ocean Drive. Bayshore Golf Course is within 3 miles of the ...
- [The Local House, Miami Beach (updated prices 2025) - Booking.com](https://www.booking.com/hotel/us/sense-south-beach.html)
  
  Concierge services offered during front desk hours. The Local House is the on-site restaurant at the The Local House. They offer an international tapas menu, brunch, and a full service bar. This Miami Beach hotel is steps from art galleries, boutique shopping and famous restaurants on Ocean Drive. Bayshore Golf Course is within 3 miles of the ...

---


## Interactive Search

Use the cell below to try your own search queries!

In [ ]:
# Enter your search query
your_query = ""  # Replace with your search query
search_hotels(your_query)